In [1]:
from yt_loader import load_yt

load_yt('https://www.youtube.com/watch?v=9ISVjh8mdlA')

/home/goma/miniconda3/envs/gpt2brain/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


[youtube] Extracting URL: https://www.youtube.com/watch?v=9ISVjh8mdlA
[youtube] 9ISVjh8mdlA: Downloading webpage
[youtube] 9ISVjh8mdlA: Downloading ios player API JSON
[youtube] 9ISVjh8mdlA: Downloading android player API JSON
[youtube] 9ISVjh8mdlA: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=9ISVjh8mdlA
[youtube] 9ISVjh8mdlA: Downloading webpage
[youtube] 9ISVjh8mdlA: Downloading ios player API JSON
[youtube] 9ISVjh8mdlA: Downloading android player API JSON
[youtube] 9ISVjh8mdlA: Downloading m3u8 information
[info] 9ISVjh8mdlA: Downloading 1 format(s): 251
[download] Destination: /mnt/d/ai/cognimesh/input/LangChain_Retrieval_Local_LLMs_for_Retrieval_QA_No_OpenAI_.webm
[download] 100% of   11.37MiB in 00:00:04 at 2.79MiB/s   
[ExtractAudio] Destination: /mnt/d/ai/cognimesh/input/LangChain_Retrieval_Local_LLMs_for_Retrieval_QA_No_OpenAI_.mp3


KeyboardInterrupt: 

In [2]:
from pdf_loader import load_pdf

load_pdf(r'input/nejmoa2304820.pdf')

DOI scraped: 10.1056/NEJMoa2304820
Saved text content to /mnt/d/ai/cognimesh/output/blankestijnEffectHemodiafiltrationHemodialysis2023.txt
Saved summary to /mnt/d/ai/cognimesh/output/blankestijnEffectHemodiafiltrationHemodialysis2023_summary.md
